<a href="https://colab.research.google.com/github/armandochernandez-ai/Curso-python-slava/blob/main/Clima/SMN_DIARIOS_JAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import csv
import time
import re
import os
import sys
from datetime import datetime, timedelta
from google.colab import files, drive
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

# Configuración
BASE_URL = "https://smn.conagua.gob.mx/tools/RESOURCES/Normales_Climatologicas/Diarios/jal"
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'es-MX,es;q=0.9',
}

# Variables disponibles en los datos diarios con nuevos nombres
VARIABLES_DIARIAS = ['p', 'evap', 'tmax', 'tmin']
VARIABLES_ORIGINALES = ['PRECIP', 'EVAP', 'TMAX', 'TMIN']
VARIABLE_MAPPING = dict(zip(VARIABLES_ORIGINALES, VARIABLES_DIARIAS))

# Valores que deben ser reemplazados por NA (incluyendo NULO)
VALORES_NULOS = ['', 'N/D', '-', '--', '---', '////', '****', '*****', 'NULL', 'null', 'N/A', 'n/a', 'NULO']

def setup_colab_environment():
    """Configura el entorno de Google Colab"""
    print("🔧 Configurando entorno de Google Colab...")

    # Montar Google Drive (opcional)
    mount_drive = input("¿Deseas montar Google Drive? (s/n): ").lower().strip()
    if mount_drive == 's':
        drive.mount('/content/drive')
        print("✅ Google Drive montado correctamente")
        return "/content/drive/MyDrive/Clima_Jalisco/DATOS_DIARIOS"
    else:
        print("📁 Usando almacenamiento temporal de Colab")
        return "/content/Clima_Jalisco/DATOS_DIARIOS"

def configurar_directorios(directorio_base=None):
    """Configura los directorios de salida"""
    if directorio_base is None:
        directorio_base = setup_colab_environment()

    directorio_datos = directorio_base

    os.makedirs(directorio_datos, exist_ok=True)

    print(f"📂 Directorio de trabajo: {os.path.abspath(directorio_datos)}")
    return directorio_datos

def descargar_datos_estacion(clave):
    """Descarga los datos diarios de una estación específica"""
    url = f"{BASE_URL}/dia{clave}.txt"
    try:
        print(f"  📥 Descargando de: {url}")
        response = requests.get(url, headers=HEADERS, timeout=30)
        response.raise_for_status()

        # Verificar si el contenido es válido
        if len(response.text.strip()) < 50:
            print(f"  ⚠️ Contenido insuficiente para estación {clave}")
            return None

        return response.text
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 404:
            print(f"  ❌ Estación {clave} no encontrada (404)")
        else:
            print(f"  ❌ Error HTTP {e.response.status_code} para estación {clave}")
        return None
    except Exception as e:
        print(f"  ❌ Error descargando datos para estación {clave}: {str(e)}")
        return None

def extraer_info_basica(contenido):
    """Extrae la información básica de la estación"""
    datos = {
        'Codigo': '',
        'site': '',
        'estado': 'Jalisco',
        'municipio': '',
        'situacion': '',
        'lat': '',
        'lon': '',
        'alt': ''
    }

    lineas = contenido.split('\n')
    for linea in lineas:
        linea = linea.strip()
        if 'ESTACIÓN  :' in linea:
            datos['Codigo'] = linea.split(':')[1].strip()
        elif 'NOMBRE    :' in linea:
            datos['site'] = linea.split(':')[1].strip()
        elif 'MUNICIPIO :' in linea:
            datos['municipio'] = linea.split(':')[1].strip()
        elif 'SITUACIÓN :' in linea:
            datos['situacion'] = linea.split(':')[1].strip()
        elif 'LATITUD   :' in linea:
            datos['lat'] = linea.split(':')[1].replace('°', '').strip()
        elif 'LONGITUD  :' in linea:
            datos['lon'] = linea.split(':')[1].replace('°', '').strip()
        elif 'ALTITUD   :' in linea:
            datos['alt'] = linea.split(':')[1].replace('msnm', '').strip()

    return datos

def limpiar_valor(valor):
    """Limpia y valida los valores, reemplazando nulos por NA"""
    if valor is None:
        return 'NA'

    valor_str = str(valor).strip()

    # Si está vacío o en la lista de valores nulos (incluyendo NULO)
    if not valor_str or valor_str in VALORES_NULOS:
        return 'NA'

    # Si son solo asteriscos o caracteres no válidos
    if re.match(r'^[*\-/\s]+$', valor_str):
        return 'NA'

    return valor_str

def convertir_fecha(fecha_str):
    """Convierte diferentes formatos de fecha a objeto datetime"""
    # Intentar formato YYYY-MM-DD
    try:
        return datetime.strptime(fecha_str, '%Y-%m-%d')
    except ValueError:
        pass

    # Intentar formato DD/MM/YYYY
    try:
        return datetime.strptime(fecha_str, '%d/%m/%Y')
    except ValueError:
        pass

    # Intentar formato D/M/YYYY
    try:
        return datetime.strptime(fecha_str, '%d/%m/%Y')
    except ValueError:
        pass

    print(f"    ⚠️ Advertencia: No se pudo parsear la fecha: {fecha_str}")
    return None

def completar_fechas_faltantes(datos_diarios):
    """Completa las fechas faltantes en los datos con registros NA"""
    if not datos_diarios:
        return []

    # Convertir fechas y ordenar
    datos_con_fechas = []
    for registro in datos_diarios:
        fecha_dt = convertir_fecha(registro['date'])
        if fecha_dt:
            datos_con_fechas.append((fecha_dt, registro))

    if not datos_con_fechas:
        return datos_diarios

    datos_con_fechas.sort(key=lambda x: x[0])

    # Encontrar rango de fechas
    fecha_inicio = datos_con_fechas[0][0]
    fecha_fin = datos_con_fechas[-1][0]

    print(f"    📅 Rango de fechas: {fecha_inicio.strftime('%Y-%m-%d')} a {fecha_fin.strftime('%Y-%m-%d')}")

    # Crear conjunto de fechas existentes
    fechas_existentes = {fecha for fecha, _ in datos_con_fechas}

    # Generar todas las fechas en el rango
    fecha_actual = fecha_inicio
    datos_completos = []

    while fecha_actual <= fecha_fin:
        if fecha_actual in fechas_existentes:
            # Buscar el registro existente
            for fecha_dt, registro in datos_con_fechas:
                if fecha_dt == fecha_actual:
                    datos_completos.append(registro)
                    break
        else:
            # Crear registro con NA para fecha faltante
            registro_na = {
                'date': fecha_actual.strftime('%Y-%m-%d'),
                'p': 'NA',
                'evap': 'NA',
                'tmax': 'NA',
                'tmin': 'NA'
            }
            datos_completos.append(registro_na)

        fecha_actual += timedelta(days=1)

    print(f"    ✅ Después de completar fechas: {len(datos_completos)} registros")
    return datos_completos

def estandarizar_formato_fecha(datos_diarios):
    """Estandariza el formato de fecha a YYYY-MM-DD"""
    for registro in datos_diarios:
        fecha_dt = convertir_fecha(registro['date'])
        if fecha_dt:
            registro['date'] = fecha_dt.strftime('%Y-%m-%d')
    return datos_diarios

def parsear_datos_diarios(contenido):
    """Extrae los datos diarios del contenido, manejando valores NULO"""
    lineas = contenido.split('\n')
    datos_diarios = []

    # Bandera para indicar cuando empezar a leer datos
    empezar_datos = False
    patron_fecha_guion = re.compile(r'^\d{4}-\d{1,2}-\d{1,2}')  # Formato YYYY-MM-DD
    patron_fecha_slash = re.compile(r'^\d{1,2}/\d{1,2}/\d{4}')  # Formato DD/MM/YYYY

    # Buscar la línea de encabezados
    for i, linea in enumerate(lineas):
        linea = linea.strip()

        # Buscar el inicio de la tabla de datos
        if 'FECHA' in linea and any(var in linea for var in VARIABLES_ORIGINALES):
            empezar_datos = True
            # Saltar la siguiente línea (unidades) si existe
            continue

        # Si estamos en la sección de datos, procesar línea
        if empezar_datos and linea:
            # Verificar si es una línea de datos (comienza con fecha en formato YYYY-MM-DD o DD/MM/YYYY)
            if patron_fecha_guion.match(linea) or patron_fecha_slash.match(linea):
                # Dividir por tabs (parece que están separados por tabs en el ejemplo)
                partes = linea.split('\t')

                # Si no hay tabs, intentar con espacios múltiples
                if len(partes) < 2:
                    partes = re.split(r'\s{2,}', linea)

                # Si aún no funciona, usar split simple
                if len(partes) < 2:
                    partes = linea.split()

                # Filtrar partes vacías
                partes = [p.strip() for p in partes if p.strip()]

                # Debe tener al menos la fecha
                if len(partes) >= 1:
                    fecha = partes[0]

                    # Procesar valores para cada variable
                    valores = {}

                    # Asignar valores según posición esperada
                    # Orden esperado: FECHA, PRECIP, EVAP, TMAX, TMIN
                    if len(partes) > 1:
                        valores['p'] = limpiar_valor(partes[1])
                    else:
                        valores['p'] = 'NA'

                    if len(partes) > 2:
                        valores['evap'] = limpiar_valor(partes[2])
                    else:
                        valores['evap'] = 'NA'

                    if len(partes) > 3:
                        valores['tmax'] = limpiar_valor(partes[3])
                    else:
                        valores['tmax'] = 'NA'

                    if len(partes) > 4:
                        valores['tmin'] = limpiar_valor(partes[4])
                    else:
                        valores['tmin'] = 'NA'

                    # Crear registro
                    registro = {
                        'date': fecha,
                        **valores
                    }
                    datos_diarios.append(registro)

            # Si encontramos líneas que indican el final de los datos
            elif any(palabra in linea.upper() for palabra in ['PROMEDIO', 'TOTAL', 'MEDIA', 'FIN DE DATOS']):
                break

    # Si no encontramos datos con el método anterior, intentar método más flexible
    if not datos_diarios:
        datos_diarios = parsear_datos_flexible(contenido)

    print(f"    📊 Se extrajeron {len(datos_diarios)} registros diarios")

    # Completar fechas faltantes
    datos_diarios = estandarizar_formato_fecha(datos_diarios)
    datos_diarios = completar_fechas_faltantes(datos_diarios)

    # Depuración: mostrar primeros 3 registros si hay datos
    if datos_diarios and len(datos_diarios) > 0:
        print(f"    📍 Primer registro: {datos_diarios[0]}")
        if len(datos_diarios) > 1:
            print(f"    📍 Último registro: {datos_diarios[-1]}")

    return datos_diarios

def parsear_datos_flexible(contenido):
    """Método más flexible para extraer datos cuando el método principal falla"""
    lineas = contenido.split('\n')
    datos_diarios = []

    # Patrones de fecha
    patron_fecha_guion = re.compile(r'^\d{4}-\d{1,2}-\d{1,2}')  # YYYY-MM-DD
    patron_fecha_slash = re.compile(r'^\d{1,2}/\d{1,2}/\d{4}')   # DD/MM/YYYY

    # Buscar cualquier línea que empiece con fecha
    for linea in lineas:
        linea = linea.strip()

        # Verificar si es una línea de datos (comienza con fecha)
        if patron_fecha_guion.match(linea) or patron_fecha_slash.match(linea):
            # Dividir por cualquier tipo de espacio (tabs, espacios múltiples)
            partes = re.split(r'\t|\s{2,}', linea)

            # Si no funciona, usar split simple
            if len(partes) < 2:
                partes = linea.split()

            # Filtrar partes vacías
            partes = [p.strip() for p in partes if p.strip()]

            if len(partes) >= 1:
                fecha = partes[0]
                valores = {}

                # Mapear las variables por posición
                for i, var in enumerate(VARIABLES_DIARIAS):
                    if i + 1 < len(partes):
                        valores[var] = limpiar_valor(partes[i + 1])
                    else:
                        valores[var] = 'NA'

                registro = {'date': fecha, **valores}
                datos_diarios.append(registro)

    return datos_diarios

def guardar_datos_estacion(directorio_salida, estacion_info, datos_diarios):
    """Guarda los datos diarios de una estación en archivo CSV"""
    if not datos_diarios:
        print(f"  ⚠️ No hay datos diarios para guardar de la estación {estacion_info['Codigo']}")
        return False

    # Crear nombre de archivo
    archivo_csv = os.path.join(directorio_salida, f"diarios_{estacion_info['Codigo']}.csv")

    # Campos del CSV con nuevos nombres
    campos = ['Codigo', 'site', 'municipio', 'lat', 'lon', 'alt', 'date'] + VARIABLES_DIARIAS

    try:
        # Escribir datos
        with open(archivo_csv, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=campos)
            writer.writeheader()

            for registro in datos_diarios:
                fila = {
                    'Codigo': estacion_info['Codigo'],
                    'site': estacion_info['site'],
                    'municipio': estacion_info['municipio'],
                    'lat': estacion_info['lat'],
                    'lon': estacion_info['lon'],
                    'alt': estacion_info['alt'],
                    'date': registro['date'],
                    'p': registro.get('p', 'NA'),
                    'evap': registro.get('evap', 'NA'),
                    'tmax': registro.get('tmax', 'NA'),
                    'tmin': registro.get('tmin', 'NA')
                }
                writer.writerow(fila)

        print(f"  💾 Datos guardados en: {archivo_csv}")
        return True
    except Exception as e:
        print(f"  ❌ Error guardando archivo para estación {estacion_info['Codigo']}: {str(e)}")
        return False

def guardar_datos_unidos(directorio_salida, estacion_info, datos_diarios):
    """Guarda todos los datos en un archivo CSV unificado"""
    if not datos_diarios:
        return

    archivo_unido = os.path.join(directorio_salida, "datos_diarios_unidos.csv")

    # Campos del CSV con nuevos nombres
    campos = ['Codigo', 'site', 'municipio', 'lat', 'lon', 'alt', 'date'] + VARIABLES_DIARIAS

    try:
        # Verificar si el archivo existe para no sobrescribir el encabezado
        modo = 'a' if os.path.exists(archivo_unido) else 'w'

        with open(archivo_unido, modo, newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=campos)

            if modo == 'w':
                writer.writeheader()
                print(f"  📦 Creando archivo unificado: {archivo_unido}")

            for registro in datos_diarios:
                fila = {
                    'Codigo': estacion_info['Codigo'],
                    'site': estacion_info['site'],
                    'municipio': estacion_info['municipio'],
                    'lat': estacion_info['lat'],
                    'lon': estacion_info['lon'],
                    'alt': estacion_info['alt'],
                    'date': registro['date'],
                    'p': registro.get('p', 'NA'),
                    'evap': registro.get('evap', 'NA'),
                    'tmax': registro.get('tmax', 'NA'),
                    'tmin': registro.get('tmin', 'NA')
                }
                writer.writerow(fila)
    except Exception as e:
        print(f"  ❌ Error actualizando archivo unificado: {str(e)}")

def procesar_estacion(clave, directorio_salida):
    """Procesa una estación individual"""
    print(f"🔍 Procesando estación: {clave}")

    contenido = descargar_datos_estacion(clave)

    if not contenido:
        print(f"  ❌ No se pudieron obtener datos para la estación {clave}")
        return False

    # Extraer información básica de la estación
    estacion_info = extraer_info_basica(contenido)

    # Si no se pudo extraer la clave, usar la clave proporcionada
    if not estacion_info['Codigo']:
        estacion_info['Codigo'] = clave

    print(f"  🏢 Estación: {estacion_info['site']} ({estacion_info['municipio']})")

    # Extraer datos diarios
    datos_diarios = parsear_datos_diarios(contenido)

    if datos_diarios:
        # Guardar en archivo individual de la estación
        guardar_datos_estacion(directorio_salida, estacion_info, datos_diarios)

        # Guardar en archivo unificado
        guardar_datos_unidos(directorio_salida, estacion_info, datos_diarios)
        return True
    else:
        print(f"  ⚠️ No se encontraron datos diarios para esta estación")
        return False

    time.sleep(1)  # Espera entre descargas para no saturar el servidor

def leer_archivo_estaciones(nombre_archivo):
    """Lee el archivo de estaciones con manejo de codificaciones"""
    try:
        with open(nombre_archivo, 'r', encoding='utf-8') as f:
            lineas = [linea.strip() for linea in f.readlines() if linea.strip()]
            print(f"📖 Se leyeron {len(lineas)} estaciones del archivo: {nombre_archivo}")
            return lineas
    except UnicodeDecodeError:
        try:
            with open(nombre_archivo, 'r', encoding='latin-1') as f:
                lineas = [linea.strip() for linea in f.readlines() if linea.strip()]
                print(f"📖 Se leyeron {len(lineas)} estaciones del archivo (codificación latin-1): {nombre_archivo}")
                return lineas
        except Exception as e:
            print(f"❌ Error leyendo archivo de estaciones {nombre_archivo}: {str(e)}")
            return []
    except FileNotFoundError:
        print(f"❌ Archivo no encontrado: {nombre_archivo}")
        return []
    except Exception as e:
        print(f"❌ Error leyendo archivo de estaciones {nombre_archivo}: {str(e)}")
        return []

def upload_estaciones_file():
    """Permite subir un archivo de estaciones en Colab"""
    print("\n📤 Subir archivo de estaciones")
    print("=" * 50)
    uploaded = files.upload()

    for filename, content in uploaded.items():
        if filename.endswith('.txt'):
            # Save in the default directory where other files are saved temporarily
            with open('estaciones.txt', 'wb') as f:
                f.write(content)
            print(f"✅ Archivo '{filename}' guardado como 'estaciones.txt'")
            return 'estaciones.txt'

    print("❌ No se subió ningún archivo .txt válido")
    return None

def solicitar_clave_estacion():
    """Solicita al usuario una clave de estación cuando no existe el archivo"""
    print("\n" + "="*60)
    print("📋 ARCHIVO DE ESTACIONES NO ENCONTRADO")
    print("="*60)
    print("El archivo 'estaciones.txt' no existe o está vacío en la ubicación esperada.")
    print("Puede:")
    print("1. Subir un archivo de estaciones")
    print("2. Proporcionar una clave de estación individual")
    print("3. Usar estaciones por defecto")
    print("\nEjemplos de claves de estación en Jalisco:")
    print("  14001, 14002, 14003, 14004, 14005, etc.")

    while True:
        print("\nOpciones:")
        print("1. Subir archivo de estaciones")
        print("2. Ingresar clave manualmente")
        print("3. Usar estaciones por defecto")
        print("4. Salir")

        opcion = input("\nSeleccione una opción (1-4): ").strip()

        if opcion == '1':
            archivo = upload_estaciones_file()
            if archivo:
                return leer_archivo_estaciones(archivo)
        elif opcion == '2':
            clave = input("Ingrese la clave de estación: ").strip()
            if clave and clave.isdigit():
                return [clave]
            else:
                print("❌ Error: La clave debe ser un número.")
        elif opcion == '3':
            print("✅ Usando estaciones por defecto de Jalisco")
            return ['14001', '14002', '14003', '14004', '14005',
                   '14006', '14007', '14008', '14009', '14010',
                   '14011', '14012', '14013', '14014', '14015']
        elif opcion == '4':
            return None
        else:
            print("❌ Opción no válida. Intente nuevamente.")

def generar_reporte_estaciones(directorio_salida):
    """Genera un reporte de las estaciones procesadas"""
    archivos = [f for f in os.listdir(directorio_salida) if f.startswith('diarios_') and f.endswith('.csv')]

    reporte = []
    for archivo in archivos:
        clave = archivo.replace('diarios_', '').replace('.csv', '')
        ruta_archivo = os.path.join(directorio_salida, archivo)

        try:
            with open(ruta_archivo, 'r', encoding='utf-8') as f:
                lineas = f.readlines()
                if len(lineas) > 1:  # Tiene encabezado y al menos un dato
                    num_registros = len(lineas) - 1
                    reporte.append({
                        'Codigo': clave,
                        'archivo': archivo,
                        'registros': num_registros
                    })
        except:
            continue

    # Guardar reporte
    if reporte:
        archivo_reporte = os.path.join(directorio_salida, "reporte_estaciones.csv")
        with open(archivo_reporte, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['Codigo', 'archivo', 'registros'])
            writer.writeheader()
            writer.writerows(reporte)

        print(f"\n📊 Reporte generado: {archivo_reporte}")
        print(f"📈 Total de estaciones con datos: {len(reporte)}")

        # Mostrar resumen
        for est in reporte:
            print(f"  📍 {est['Codigo']}: {est['registros']} registros")

        return len(reporte)
    return 0

def descargar_archivos_resultado(directorio_salida):
    """Permite descargar los archivos resultantes en Colab"""
    print("\n📥 DESCARGAR RESULTADOS")
    print("=" * 40)

    archivos_descargables = []

    # Buscar archivos CSV en el directorio
    for archivo in os.listdir(directorio_salida):
        if archivo.endswith('.csv'):
            archivos_descargables.append(os.path.join(directorio_salida, archivo))

    if archivos_descargables:
        print("Archivos disponibles para descargar:")
        for i, archivo in enumerate(archivos_descargables, 1):
            print(f"  {i}. {os.path.basename(archivo)}")

        descargar = input("\n¿Desea descargar todos los archivos en su computadora? (s/n): ").lower().strip()
        if descargar == 's':
            # Crear archivo zip con todos los resultados
            import zipfile
            zip_path = os.path.join(directorio_salida, "resultados_clima.zip")

            with zipfile.ZipFile(zip_path, 'w') as zipf:
                for archivo in archivos_descargables:
                    zipf.write(archivo, os.path.basename(archivo))

            print("📦 Descargando archivo comprimido...")
            files.download(zip_path)
            print("✅ Descarga completada")
    else:
        print("❌ No hay archivos para descargar")

def mostrar_resumen_estadisticas(directorio_salida):
    """Muestra un resumen estadístico de los datos"""
    archivo_unido = os.path.join(directorio_salida, "datos_diarios_unidos.csv")

    if os.path.exists(archivo_unido):
        print("\n📊 RESUMEN ESTADÍSTICO")
        print("=" * 40)

        try:
            # Use low_memory=False to avoid DtypeWarning for mixed types
            df = pd.read_csv(archivo_unido, low_memory=False)
            print(f"Total de registros: {len(df)}")
            print(f"Total de estaciones: {df['Codigo'].nunique()}")

            # Convert date column to datetime objects, coercing errors
            df['date'] = pd.to_datetime(df['date'], errors='coerce')
            df = df.dropna(subset=['date']) # Remove rows where date conversion failed

            if not df.empty:
                print(f"Rango de fechas: {df['date'].min().strftime('%Y-%m-%d')} a {df['date'].max().strftime('%Y-%m-%d')}")
            else:
                 print("No valid dates found in the data.")


            # Estadísticas por variable - convert to numeric, coercing errors
            variables = ['p', 'evap', 'tmax', 'tmin']
            for var in variables:
                # Convert to numeric, invalid parsing will be set as NaN
                df[var] = pd.to_numeric(df[var], errors='coerce')
                datos_validos = df[var].dropna() # Drop NaN values

                if len(datos_validos) > 0:
                    print(f"\n{var.upper()}:")
                    print(f"  Registros válidos: {len(datos_validos)}")
                    print(f"  Promedio: {datos_validos.mean():.2f}")
                    print(f"  Mínimo: {datos_validos.min():.2f}")
                    print(f"  Máximo: {datos_validos.max():.2f}")
                else:
                    print(f"\n{var.upper()}: No valid numeric data found.")


        except Exception as e:
            print(f"❌ Error generando estadísticas: {e}")
    else:
        print(f"\n📊 RESUMEN ESTADÍSTICO: Archivo unificado '{archivo_unido}' no encontrado.")

def main():
    print("🌤️ SISTEMA DE EXTRACCIÓN DE DATOS METEOROLÓGICOS DIARIOS DE JALISCO")
    print("=" * 70)
    print("Variables: date, p, evap, tmax, tmin")
    print("Valores nulos (incluyendo 'NULO') se reemplazan por: NA")
    print("Se completarán fechas faltantes con registros NA")

    # Configurar directorios
    directorio_salida = configurar_directorios()

    # Determine the expected path for estaciones.txt
    # If Google Drive is mounted, check the specified path
    if "/content/drive" in directorio_salida: # Simple check if Drive was likely mounted
        archivo_entrada = os.path.join(directorio_salida, 'estaciones.txt')
    else:
        # Otherwise, check the default temporary location
        archivo_entrada = 'estaciones.txt'

    estaciones = []

    print(f"\n🔎 Buscando archivo de estaciones en: {archivo_entrada}")
    if os.path.exists(archivo_entrada) and os.path.getsize(archivo_entrada) > 0:
        estaciones = leer_archivo_estaciones(archivo_entrada)
    else:
        print(f"📄 Archivo {archivo_entrada} no encontrado o está vacío")
        estaciones = solicitar_clave_estacion()

        # If the user didn't provide valid input
        if not estaciones:
            print("❌ No se proporcionó una clave de estación válida. Saliendo.")
            return

    if estaciones:
        total = len(estaciones)
        print(f"\n🔄 Procesando {total} estaciones...")

        exitosas = 0
        for i, clave in enumerate(estaciones, 1):
            print(f"\n[{i}/{total}] ", end="")
            if procesar_estacion(clave, directorio_salida):
                exitosas += 1
            time.sleep(1)  # Espera entre descargas para no saturar el servidor

        print(f"\n🎉 ¡PROCESO COMPLETADO!")
        print(f"✅ Estaciones con datos extraídos: {exitosas}/{total}")
        print(f"📂 Datos guardados en: {directorio_salida}")

        # Generar reporte
        generar_reporte_estaciones(directorio_salida)

        # Mostrar estadísticas
        mostrar_resumen_estadisticas(directorio_salida)

        # Ofrecer descargar resultados
        descargar_archivos_resultado(directorio_salida)

    else:
        print("❌ No se encontraron estaciones para procesar")

if __name__ == "__main__":
    main()

🌤️ SISTEMA DE EXTRACCIÓN DE DATOS METEOROLÓGICOS DIARIOS DE JALISCO
Variables: date, p, evap, tmax, tmin
Valores nulos (incluyendo 'NULO') se reemplazan por: NA
Se completarán fechas faltantes con registros NA
🔧 Configurando entorno de Google Colab...
¿Deseas montar Google Drive? (s/n): s
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive montado correctamente
📂 Directorio de trabajo: /content/drive/MyDrive/Clima_Jalisco/DATOS_DIARIOS

🔎 Buscando archivo de estaciones en: /content/drive/MyDrive/Clima_Jalisco/DATOS_DIARIOS/estaciones.txt
📖 Se leyeron 275 estaciones del archivo: /content/drive/MyDrive/Clima_Jalisco/DATOS_DIARIOS/estaciones.txt

🔄 Procesando 275 estaciones...

[1/275] 🔍 Procesando estación: 14001
  📥 Descargando de: https://smn.conagua.gob.mx/tools/RESOURCES/Normales_Climatologicas/Diarios/jal/dia14001.txt
  🏢 Estación: ACATIC (SMN) (ACATIC)
    📊 Se extrajeron 4429 registros diar

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Descarga completada
